In [1]:
# logical change of basis
# example: in dual-rail 1Q gates are created by 2Q interactions

# |0>L = |01>, |1>L = |10>
# |e1> = |00>, |e2> = |11>

# Hadamard
# as a density matrix, includes implicit error states stay in error states
# H = 1/2 [|0><0|L + |0><1|L + |1><0|L - |1><1|L] (+ |e1><e1| + |e2><e2|)

# becomes
# = 1/2 [|01><01| + |01><10| + |10><01| - |10><10|] + |00><00| + |11><11|

# as an operator
# = [[1, 0, 0, 0], [0,1/sqrt(2),1/sqrt(2),0], [0,1/sqrt(2), -1/sqrt(2),0], [0,0,0,1]]

# convert to weylchamber coordinate
# -> (1/2, 1/4, 1/4)

# repeat....
# for some encoding, define some map between nQ -> (n+1)Q gates.

In [ ]:
# including error states
# phase flips errors E = |-+><-+| + |+-><+-|
# ancilla qubits (x, y) for distinguishing between the two types of errors

# NOTE is this form of an Isometry valid?
# idea is that it is simpler to leave out the error states until they are actually needed
# basically, the no error flag qubit is implicit
# U = |g+f, g+f><0| + |g-f, g-f><1| + |++x><+-| + |--y><-+|

In [72]:
from quantum_logical.basis import DualRail

encoding = DualRail()
encoding.ancilla[0].detection_operator

(Quantum object: dims = [[2, 2], [1, 1]], shape = (4, 1), type = ket
Qobj data =
[[1.]
 [0.]
 [0.]
 [0.]],)


Quantum object: dims = [[2, 2, 2], [2, 2, 2]], shape = (8, 8), type = oper, isherm = False
Qobj data =
[[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]